<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Fall2021/blob/main/selenium_scraping_Antoine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 40 not upgraded.
Need to get 86.0 MB of archives.
After this operation, 298 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 91.0.4472.101-0ubuntu0.18.04.1 [1,124 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 91.0.4472.101-0ubuntu0.18.04.1 [76.1 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 91.0.4472.101-0ubuntu0.1

In [ ]:
from selenium import webdriver
from lxml import html
import jax
import jax.numpy as jnp
import numpy as np
from scipy.optimize import minimize
from plotly.subplots import make_subplots
import plotly.io as pio
from collections import deque
pio.templates.default='plotly_dark'

#JAX default is 32 bit - enable 64 bit - double precision
from jax.config import config
config.update("jax_enable_x64", True)

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [ ]:
wd = webdriver.Chrome('chromedriver',options=options)
wd.get('http://www.ddbst.com/en/EED/PCP/VAP_C174.php')
tree=html.fromstring(wd.page_source)

In [ ]:
T_strings=tree.xpath('//*[@id="online-ddb-form"]/div[2]/table[3]/tbody/tr/td[1]/text()')
P_strings=tree.xpath('//*[@id="online-ddb-form"]/div[2]/table[3]/tbody/tr/td[2]/text()')
data = np.array([T_strings, P_strings]).astype(np.float64)

In [ ]:
T=data[0,:]
log_Pv = np.log10(1000*data[1,:])

In [ ]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=T, y=log_Pv, mode='markers', marker_size=8, marker_color='rgba(0,0,0,0)',
                marker_line_color='rgb(0,0,255)', marker_line_width=1, name='Data')
fig.update_layout(width=800, height=500)
fig.update_layout(xaxis_title='T (K)', yaxis_title='$Log_{10} P\ (Pa)$a')
fig.show()

In [ ]:
def model(a, b, c, T):
  return a-b/(T+c)

In [ ]:
a_guess = 10.
b_guess = 2000.0
c_guess = 0.0
x0 = [a_guess, b_guess, c_guess]

def sqerror(x, T, log_Pv):
  model_Pv = model(x[0],x[1],x[2], T)
  err = jnp.sum( (model_Pv-log_Pv)**2)
  return err

# res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10)
res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10, jac=jax.grad(sqerror))

In [ ]:

T_model = jnp.linspace(T[0],T[-1],100)
fig.add_scatter(x=T_model, y=model(*res.x, T_model), mode='lines', line_width=1, name='Antoine')
fig.show()